In [1]:
from sympy import symbols, init_printing, S, Derivative, diff, simplify, solve, lambdify, cos, sin
from sympy.physics.vector import vlatex
from sympy.abc import a, b, c, d
import numpy as np
import scipy.integrate as integrate
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='html5')

init_printing(latex_printer=vlatex, latex_mode='inline')

In [2]:
t = symbols('t')

g = S(9.8)
l = [S(1.0)] * 2
m = [S(1.0)] * 2
r = [temp_l/2 for temp_l in l]
I = [(temp_m * temp_l**2)/12 for temp_m,temp_l in zip(m,l)]

theta = [w(t) for w in symbols('theta0:2')]
theta_dot = [Derivative(w, t) for w in theta]
theta_ddot = [Derivative(w, t, t) for w in theta]

In [3]:
x = [None] * 2
y = [None] * 2
x_dot = [None] * 2
y_dot = [None] * 2

x[0] = r[0] * cos(theta[0])
y[0] = r[0] * sin(theta[0])
x[1] = l[1] * cos(theta[0]) + r[1] * cos(theta[0] + theta[1])
y[1] = l[1] * sin(theta[0]) + r[1] * sin(theta[0] + theta[1])

x_dot[0] = diff(x[0], t)
y_dot[0] = diff(y[0], t)
x_dot[1] = diff(x[1], t)
y_dot[1] = diff(y[1], t)

In [4]:
kinetic = (m[0] * (x_dot[0] ** 2 + y_dot[0] ** 2)
           + m[1] * (x_dot[1] ** 2 + y_dot[1] ** 2)
           + I[0] * (theta_dot[0]) ** 2
           + I[1] * (theta_dot[0] + theta_dot[1]) ** 2) / 2

potential = (m[0] * g * y[0]) + (m[1] * g * y[1])

lagrange = kinetic - potential

$$x_0= r_0 \cos{(\theta_0)}$$
$$y_0= r_0 \sin{(\theta_0)}$$
$$x_1= l_0 \cos{(\theta_0)} + r_1 \cos{(\theta_0 + \theta_1)}$$
$$y_1= l_0 \sin{(\theta_0)} + r_1 \sin{(\theta_0 + \theta_1)}$$

$$\dot{x_0}= -r_0 \sin{(\theta_0)}\dot{\theta_0}$$
$$\dot{y_0}= r_0 \cos{(\theta_0)}\dot{\theta_0}$$
$$\dot{x_1}= -(l_0 \sin{(\theta_0)} + r_1 \sin{(\theta_0 + \theta_1)})\dot{\theta_0} - r_1\sin{(\theta_0 + \theta_1)}\dot{\theta_1}$$
$$\dot{y_1}= (l_0 \cos{(\theta_0)} + r_1 \cos{(\theta_0 + \theta_1)})\dot{\theta_0} + r_1\cos{(\theta_0 + \theta_1)}\dot{\theta_1}$$

In [5]:
L = [None] * 2
L[0] = diff(lagrange, theta_dot[0], t) - diff(lagrange, theta[0])
L[1] = diff(lagrange, theta_dot[1], t) - diff(lagrange, theta[1])

In [6]:
solution = solve(L, theta_ddot)

In [7]:
inputs = [(theta_dot[0], a), (theta[0], b), (theta_dot[1], c), (theta[1], d)]

LS = [None] * 2
theta0_ddot_function = lambdify((b, a, d, c), simplify(solution[theta_ddot[0]]).subs(inputs))
theta1_ddot_function = lambdify((b, a, d, c), simplify(solution[theta_ddot[1]]).subs(inputs))

In [8]:
def double_pendulum_deriv(this_state, time_step):
    this_theta0, this_theta0_dot, this_theta1, this_theta1_dot = this_state

    next_theta0_dot = this_theta0_dot
    next_theta1_dot = this_theta1_dot

    next_theta0_ddot = theta0_ddot_function(*this_state)
    next_theta1_ddot = theta1_ddot_function(*this_state)

    return np.array([next_theta0_dot, next_theta0_ddot, next_theta1_dot, next_theta1_ddot])

In [9]:
def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def animate(i):
    thisx = [0, x1_pos[i], x2_pos[i]]
    thisy = [0, y1_pos[i], y2_pos[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i * dt))
    return line, time_text

In [12]:
dt = 0.05
t = np.arange(0.0, 15, dt)

theta0_initial = 0.0
theta0_dot_initial = 0.0
theta1_initial = 0.0
theta1_dot_initial = 0.0

initial_state = np.radians([theta0_initial, theta0_dot_initial, theta1_initial, theta1_dot_initial])

pos = integrate.odeint(double_pendulum_deriv, initial_state, t)

x1_pos = float(l[0]) * np.cos(pos[:, 0])
y1_pos = float(l[0]) * np.sin(pos[:, 0])

x2_pos = x1_pos + float(l[1]) * np.cos(pos[:, 0] + pos[:, 2])
y2_pos = y1_pos + float(l[1]) * np.sin(pos[:, 0] + pos[:, 2])

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.grid()
ax.set_aspect('equal', adjustable='box')

line, = ax.plot([], [], 'k-', lw=4, solid_capstyle='round')
time_template = 'time = %.2fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

ani = animation.FuncAnimation(fig, animate, frames=len(pos),
                              interval=25, blit=True, init_func=init)

In [13]:
HTML(ani.to_html5_video())